In [2]:
# ref: https://files.xisupport.com/other_files/JupyterNotebook/Standa_8SMC5_USB_Python_tutorial.html
import pathlib
import os
import time
import libximc.highlevel as ximc

# Virtual device creation

In [3]:
virtual_device_filename = "virtual_motor_controller_1.bin"
virtual_device_file_path = os.path.join(pathlib.Path().cwd(), virtual_device_filename)

# Real devices search

In [4]:
# Devices search
devices = ximc.enumerate_devices(ximc.EnumerateFlags.ENUMERATE_NETWORK | ximc.EnumerateFlags.ENUMERATE_PROBE)

if len(devices) == 0:
    print("The real devices were not found. A virtual device will be used.")
else:
    # Print real devices list
    print("Found {} real device(s):".format(len(devices)))
    for device in devices:
        print("  {}".format(device))

# Device URI setup
Each motor controller in the system should have a unique URI. Use such URI to connect to the specific device.

There are several URI types:

- xi-emu:///<abs_path_to_file> - for virtual devices.
- xi-com:\\.\COM<N> - for local devices connected via USB.
- xi-tcp://<IP>:<Port> - for network devices with Ethernet support.
- xi-net://<IP>/serial - for network devices connected through Standa 8Eth1 converter.
    
To use real device, change URI in the cell below. In case you don’t have any real devices, execute the cell without URI modification.

In [4]:
device_uri = "xi-emu:///{}".format(virtual_device_file_path)
# device_uri = r"xi-com:\\.\COM29"
# device_uri = "xi-tcp://172.16.131.140:1820"
# device_uri = "xi-net://192.168.1.120/abcd"

# First steps

In the beginning you should create Axis instance using desired URI. As a result you will get an object (in our example it's called axis) through which you will be able to interact with the device.

The connection is closed automatically when the garbage collector destructs Axis object. But of course you can close the connection manually via axis.close_device().

In [5]:
axis = ximc.Axis(device_uri)
# To open the connection, you must manually call `open_device()` method
axis.open_device()

In [6]:
# get_position method returns position_t object
position = axis.get_position()
print("Initial position:", position.Position)

print("Start moving")
axis.command_right()
for i in range(3):
    time.sleep(1)
    print("Moving...")

print("Stop moving")
axis.command_stop()

position = axis.get_position()
print("Final position:", position.Position)

Initial position: 0
Start moving
Moving...
Moving...
Moving...
Stop moving
Final position: 3045


In [7]:
axis.close_device()
print("Device disconnected")

Device disconnected


# Moving to a specific coordinates

You can use command_move() and command_movr() methods to move axis to a specific absolute or relative position respectively.

The following sequence of absolute and relative movements will be performed in the cell below.

In [8]:
axis.open_device()

# ==== Set current position as zero ====
axis.command_zero()

# Object instances should be passed by reference
position = axis.get_position()
print("Initial position:", position.Position)


# ==== Move to the first absolute position (X = 100) ====
next_position = 100
print("Move to position:", next_position)
axis.command_move(next_position, 0)

print("Moving...")
axis.command_wait_for_stop(100)

position = axis.get_position()
print("Current position:", position.Position)


# ==== Move to the second absolute position (X = 50) ====
next_position = 50
print("Move to position:", next_position)
axis.command_move(next_position, 0)

print("Moving...")
axis.command_wait_for_stop(100)

position = axis.get_position()
print("Current position:", position.Position)


# ==== Perform a relative shift by 100 ====
relative_shift = 100
print("Perform a relative shift by", relative_shift)
print("So we are going to", position.Position, "+", relative_shift, " =", position.Position + relative_shift)
axis.command_movr(relative_shift, 0)

print("Moving...")
axis.command_wait_for_stop(100)

position = axis.get_position()
print("Current position:", position.Position)


# ==== Perform a relative shift by -150 ====
relative_shift = -150
print("Perform a relative shift by", relative_shift)
print("So we are going to", position.Position, "+ (", relative_shift, ") =", position.Position + relative_shift)
axis.command_movr(relative_shift, 0)

print("Moving...")
axis.command_wait_for_stop(100)

position = axis.get_position()
print("Current position:", position.Position)

axis.close_device()
print("Done")

Initial position: 0
Move to position: 100
Moving...
Current position: 100
Move to position: 50
Moving...
Current position: 50
Perform a relative shift by 100
So we are going to 50 + 100  = 150
Moving...
Current position: 150
Perform a relative shift by -150
So we are going to 150 + ( -150 ) = 0
Moving...
Current position: 0
Done


# Using user units
By default all values (position, speed, acceleration…) are represented in motor steps and microsteps. You can use more convenient custom units such as millimeters, inches, degrees, radians by calling _calb postfixed methods.

To use user units you should know conversion coefficient in units per step. In most cases you can find such coefficient in specification for your stage.

For example [8MT193-100 Motorized Linear Stage](https://www.standa.lt/products/catalog/motorised_positioners?item=62&prod=motorized_linear_stage) has 2.5 µm / step resolution. In case you would like to use millimeters as user units, the conversion coefficient will be 0.0025 mm / step.



In [9]:
axis.open_device()

# ==== User unit setup ====
# We will use mm as user units
# In our example conversion coefficient will be 0.0025 mm / step.
# Set conversion coefficient for your stage here if needed
step_to_mm_conversion_coeff = 0.0025  # mm / step

# Get information about microstep mode
engine_settings = axis.get_engine_settings()

# Now we can set calibration settings for our axis
axis.set_calb(step_to_mm_conversion_coeff, engine_settings.MicrostepMode)

# ==== Perform a shift by using user units (mm in our case) ====
position_calb = axis.get_position_calb()
print("Current position:", position_calb.Position, "mm")

next_position_in_mm = 5.21
print("Move to position:", next_position_in_mm, "mm")
axis.command_move_calb(next_position_in_mm)

print("Moving...")
axis.command_wait_for_stop(100)

position_calb = axis.get_position_calb()
print("Current position:", position_calb.Position, "mm")

next_position_in_mm = 0
print("Move to position:", next_position_in_mm, "mm")
axis.command_move_calb(next_position_in_mm)

print("Moving...")
axis.command_wait_for_stop(100)

position_calb = axis.get_position_calb()
print("Current position:", position_calb.Position, "mm")

axis.close_device()
print("Done")

Current position: 0.0 mm
Move to position: 5.21 mm
Moving...
Current position: 5.210000038146973 mm
Move to position: 0 mm
Moving...
Current position: 0.0 mm
Done
